Подключаем необходимые в работе библиотеки/модули: json (для работы с json-форматом данных), pandas (для работы с табличными данными), requests (для работы с HTTP-запросами), matplotlib.pyplot (для создания графиков и визуализации данных), networkx (для создания, манипуляции и изучения структуры, динамики и функций сложных сетей), random (для генерации случайных чисел и выполнения случайных операций), scipy (для научных вычислений):

In [ ]:
import json
import pandas as pd
import requests
import matplotlib.pyplot as plt
import networkx as nx
import random
import scipy

Создаём класс "vk_user", атрибутами которого станут ID пользователя, его ФИО и список его друзей (их ID). Также задаём метод для представления данных в формате, удобном для сериализации в JSON:

In [ ]:
class vk_user:
    def __init__(self, vk_id, name, friends):
        self.vk_id = vk_id
        self.name = name
        self.friends = friends

    def to_dict(self):
        return {
            'vk_id': self.vk_id,
            'name': self.name,
            'friends': self.friends
        }

Определяем статический метод, который принимает аргумент, представляющий собой словарь. А также задаём функции для cериализации данных в формате JSON и десериализации JSON в объекты Python:

In [ ]:
@staticmethod
def from_dict(data):
    return vk_user(vk_id=data['vk_id'], name=data['name'], friends=data['friends'])
    

def save_data_to_json_file(friend_list, filename):
    with open(filename, 'w') as file:
        json.dump([fl.to_dict() for fl in friend_list], file)

def load_data_from_json_file(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
        return [vk_user.from_dict(f) for f in data]

В список "users" запишем данные из CSV-файла "BSMO_10_24" (удалив строки с пустыми значениями):

In [ ]:
users = []

df = pd.read_csv("BSMO_10_24.csv")
df.columns = ["name", "vk_link", "id"]
df = df.dropna()
df['id'] = df['id'].astype(int).astype(str)

for index, row in df.iterrows():
    users.append(vk_user(row["id"], row["name"], None))

Поскольку необходимо найти друзей и друзей-друзей пользователя, а также установить между всеми наличие дружеских связей (в т.ч. и между друзьями-друзей), то ставим глубину поиска равную трём, не забыв ограничить круг поиска до ста друзей на одного человека (для адекватной визуализации графа):

In [ ]:
deep = 3
max = 100

Циклом проходимся по пользователям из списка "users", используя для выгрузки необходимых данных хранимую процедуру в VK – "friends.get". За первый проход будет найден список друзей и соотвествующие дружеские связи, за второй проход — список друзей-друзей и соотвествующие дружеские связи, а третий проход лишь установит наличие дружеских связей друзей-друзей с остальными:

In [ ]:
for d in range(deep):
    new_users = []
    for user in users:
        if user.friends == None:
            url = f'https://api.vk.com/method/friends.get?user_id={user.vk_id}&&v=5.199'
            response = requests.get(url)
            if 'response' not in response.json():
                continue
            friends = response.json()['response']['items']
            for i in range(len(friends)):
                friend = friends[i]
                if i >= max:
                    break
                if any(person.vk_id == friend for person in users):
                    if user.friends == None:
                        user.friends = []
                    user.friends.append(friend)
                if d != 3 and all(person.vk_id != friend for person in users):
                    new_users.append(vk_user(friend, None, None))
    users.extend(new_users)

Сохраняем данные о пользователях в файл "friendships.json", а затем, для дальнейшего построения графа, загружаем данные в переменную "users", наполняя её Python-объектами:

In [ ]:
save_data_to_json_file(users, "friendships.json")

users = load_data_from_json_file('friendships.json')

Создаём пустой неориентированный граф "G":

In [ ]:
G = nx.Graph()

Циклом проходимся по каждому объекту в списке "users" (необходимы их ID) и добавляем вершины в граф "G":

In [ ]:
for user in users:
    G.add_node(user.vk_id)

Проходясь циклом по всем друзьям VK-пользователей, добавляем рёбра между ними в графе:

In [ ]:
for user in users:
    if user.friends != None:
        for friend_id in user.friends:
            if G.has_edge(user.vk_id, friend_id):
                continue
            G.add_edge(user.vk_id, friend_id)

Классифицируем вершины графа "G" по цветовым категориям в зависимости от заданных условий:
1) red_nodes: вершины, соответствующие одногруппникам
2) lightblue_nodes: вершины, соотвествующие пользователям с менее чем 2 друзьями
3) skyblue_nodes: вершины, соотвествующие пользователям с менее чем 10 друзьями
4) cornflowerblue_nodes: вершины, соотвествующие пользователям с менее чем 30 друзьями
5) blue_nodes: вершины, соотвествующие пользователям с менее чем 50 друзьями
6) darkblue_nodes: вершины, соотвествующие пользователям с 50 и более друзьями

In [ ]:
red_nodes = [node for node in list(G.nodes()) if all(person.vk_id != node or person.name is not None for person in users)]
other_nodes = list(set(G.nodes()) - set(red_nodes))
lightblue_nodes = list(node for node in other_nodes if all(person.vk_id != node or person.friends is None or len(person.friends) < 2 for person in users))
other_nodes = list(set(other_nodes) - set(lightblue_nodes))
skyblue_nodes = list(node for node in other_nodes if all(person.vk_id != node or len(person.friends) < 10 for person in users))
other_nodes = list(set(other_nodes) - set(skyblue_nodes))
cornflowerblue_nodes = list(node for node in other_nodes if all(person.vk_id != node or len(person.friends) < 30 for person in users))
other_nodes = list(set(other_nodes) - set(cornflowerblue_nodes))
blue_nodes = list(node for node in other_nodes if all(person.vk_id != node or len(person.friends) < 50 for person in users))
darkblue_nodes = list(set(other_nodes) - set(blue_nodes))

Задаём цвета рёбер графа в разных оттенках чёрного (для лучшей визуализации связей в готовом графе):

In [ ]:
edge_colors = ['lightgray', 'gray', 'darkgray', 'black']
random_edge_colors = [random.choice(edge_colors) for _ in range(len(G.edges()))]

Определим количество вершин и рёбер в графе "G": 

In [ ]:
print(f"Количество вершин: {len(G.nodes())}")
print(f"Количество рёбер: {len(G.edges())}")

Получили следующие значения: 112715 вершин и 301430 рёбер.

!!!Визуализируем граф "G" (с использованием алгоритма Камада-Кавай), настроив размеры вершин (и рёбер) для выделенных ранее групп:

In [ ]:
pos = nx.spring_layout(G, iterations=3)
nx.draw_networkx_nodes(G, pos, nodelist=lightblue_nodes, node_color='#87cefa33', node_size=1)
nx.draw_networkx_nodes(G, pos, nodelist=skyblue_nodes, node_color='#87ceeb33', node_size=2)
nx.draw_networkx_nodes(G, pos, nodelist=cornflowerblue_nodes, node_color='#00bfff33', node_size=3)
nx.draw_networkx_nodes(G, pos, nodelist=blue_nodes, node_color='#0000ff33', node_size=4)
nx.draw_networkx_nodes(G, pos, nodelist=darkblue_nodes, node_color='#00008b33', node_size=5)
nx.draw_networkx_nodes(G, pos, nodelist=red_nodes, node_color='red', node_size=6)
nx.draw_networkx_edges(G, pos, edge_color=random_edge_colors, width=0.4)

Задаём параметры визуализации графа и сохраняем его в файл "graph.png":

In [ ]:
plt.axis('off')
plt.figure(figsize=(15, 12))
plt.rcParams.update({'figure.dpi': 600})
plt.savefig("graph.png", dpi=600)

В итоге получили следующий граф:

![img](graph.png)

Рассмотрим, какие виды центральности в графе используем в работе:
1) Центральность по близости (closeness centrality) — определяет, насколько данная вершина близка ко всем остальным вершинам в сети (кратчайший путь).
2) Центральность по посредничеству (betweenness centrality) — показывает, насколько часто рассматриваемая вершина является "перевалочным пунктом" при переходах от одной вершины графа до любой другой.
3) Центральность по собственному вектору (eigenvector centrality) — демонстрирует зависимость между центральностью участника и центральностями его друзей. Принцип данной меры можно описать так: "если мои друзья влиятельны, то и я буду более влиятельным".

Вычислим центральность графа по близости, посредничеству и собственному значению:

In [ ]:
students_nodes = list(G.nodes())[:30]
closeness_centrality_users = sorted([(node, nx.closeness_centrality(G, u=node)) for node in students_nodes], key=lambda i: i[1], reverse=True)
betweenness_centrality_users = sorted(list(nx.betweenness_centrality(G, k=10000).items()), key=lambda i: i[1], reverse=True)
eigenvector_centrality_users = sorted(list(nx.eigenvector_centrality(G).items()), key=lambda i: i[1], reverse=True)

def print_central_student(students, way):
    for ccu, metrica in students:
        for user in users:
            if user.vk_id == ccu and user.name is not None:
                print(f"Центральный студент в группе по {way}: {user.name} (центральность {metrica})")
                return

print_central_student(closeness_centrality_users, "близости")
print_central_student(betweenness_centrality_users, "посредничеству")
print_central_student(eigenvector_centrality_users, "собственному значению")

В итоге получили следующие данные:
1. Центральный студент в группе по близости — Латынова Татьяна Алексеевна (центральность 0.2766227425182126)
2. Центральный студент в группе по посредничеству — Цыганков Никита Денисович (центральность 0.03717725651972163)
3. Центральный студент в группе по собственному значению — Латынова Татьяна Алексеевна (центральность 0.012073259831489906)